In [ ]:
!pip3 install -q ktrain

     |████████████████████████████████| 25.2MB 130kB/s 
     |████████████████████████████████| 421.8MB 41kB/s 
     |████████████████████████████████| 6.7MB 13.6MB/s 
     |████████████████████████████████| 983kB 16.9MB/s 
     |████████████████████████████████| 245kB 57.1MB/s 
     |████████████████████████████████| 675kB 49.4MB/s 
     |████████████████████████████████| 471kB 40.7MB/s 
     |████████████████████████████████| 450kB 54.4MB/s 
     |████████████████████████████████| 3.9MB 48.1MB/s 
     |████████████████████████████████| 890kB 60.3MB/s 
     |████████████████████████████████| 3.8MB 47.0MB/s 
     |████████████████████████████████| 1.1MB 51.1MB/s 
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
train_b=pd.read_csv("/content/drive/My Drive/minor/english_dataset/eng23train.csv") #training data
test_b=pd.read_csv("/content/drive/My Drive/minor/english_dataset/eng23test.csv") #testing data
x_train,x_test,y_train,y_test=train_test_split(train_b['text'],train_b['task_2'],test_size=0.15,random_state=42) #training and validation split
target_names=[0,1,2] #names of classes

In [ ]:
#convertig the tweets to list of strings and labels to list data type
x_train=list(x_train)
x_test=list(x_test)
y_train=list(y_train)
y_test=list(y_test)

In [ ]:
import ktrain
from ktrain import text
MODEL_NAME = 'xlnet-base-cased' #xlnet predefined model
t = text.Transformer(MODEL_NAME, maxlen=500, classes=target_names) #load model, text and label preprocessor for xlnet
trn = t.preprocess_train(x_train, y_train) #preprocess text and labels for training data
val = t.preprocess_test(x_test, y_test) #preprocess text and labels for validation data
model = t.get_classifier() #get xlnet classifier
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6) #get trainable learner for xlnet

/usr/local/lib/python3.6/dist-packages/ktrain/text/preprocessor.py:398: UserWarning: The class_names argument is replacing the classes argument. Please update your code.
  warnings.warn('The class_names argument is replacing the classes argument. Please update your code.')


preprocessing train...
language: en
train sequence lengths:
	mean : 25
	95percentile : 47
	99percentile : 53


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 23
	95percentile : 45
	99percentile : 52


In [ ]:
learner.fit_onecycle(5e-5, 4) #training for epochs=4 ,learning rate=5e-5



begin training using onecycle policy with max lr of 5e-05...
Train for 321 steps, validate for 11 steps
Epoch 1/4
321/321 [==============================] - 325s 1s/step - loss: 1.0167 - accuracy: 0.5154 - val_loss: 0.8910 - val_accuracy: 0.6618
Epoch 2/4
321/321 [==============================] - 312s 971ms/step - loss: 0.7922 - accuracy: 0.6736 - val_loss: 0.7793 - val_accuracy: 0.6706
Epoch 3/4
321/321 [==============================] - 312s 971ms/step - loss: 0.7079 - accuracy: 0.6903 - val_loss: 0.7446 - val_accuracy: 0.7088
Epoch 4/4
321/321 [==============================] - 312s 970ms/step - loss: 0.5488 - accuracy: 0.7611 - val_loss: 0.7559 - val_accuracy: 0.6735


In [ ]:
learner.validate() #predictions metrics for validation data

              precision    recall  f1-score   support

           0       0.74      0.74      0.74       114
           1       0.78      0.72      0.75       172
           2       0.31      0.41      0.35        54

    accuracy                           0.67       340
   macro avg       0.61      0.62      0.61       340
weighted avg       0.70      0.67      0.68       340



array([[ 84,  11,  19],
       [ 20, 123,  29],
       [  9,  23,  22]])

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc=t) #get predictor for the trained model of  xlnet

In [ ]:
test_b_text=list(test_b['text']) #get text for testing
test_b_label=list(test_b['task_2']) #get label of testing
# val = t.preprocess_test(x_test, y_test) #preprocess text and labels for validation data

In [ ]:
# predicted probability scores for each category
pred_y=predictor.predict_proba(test_b_text)

In [ ]:
pred_y=np.argmax(pred_y,axis=1) #get class with maximum probability

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(test_b_label,pred_y)) # classification report

              precision    recall  f1-score   support

           0       0.66      0.90      0.76        93
           1       0.70      0.76      0.73       124
           2       0.46      0.17      0.25        71

    accuracy                           0.66       288
   macro avg       0.61      0.61      0.58       288
weighted avg       0.63      0.66      0.62       288

